# Thrust vectored AUV

## Extremeley similar to thrust vector rocket landing (a la moda di SpaceX)
![](rocket.png)
![](equations.png)

In [1]:
from sympy import *
init_printing()

In [2]:
# state
x, y, vx, vy, theta, omega = symbols('x y vx vy theta omega', real=True)
p = Matrix([x, y])
v = Matrix([vx, vy])
s = Matrix([p, v, [theta], [omega]])
s.T

[x  y  vx  vy  θ  ω]

In [3]:
# costate variables
lx, ly, lvx, lvy, ltheta, lomega = symbols('lambda_x lambda_y lambda_v_x lambda_v_y lambda_theta lambda_omega', real=True)
lp = Matrix([lx, ly])
lv = Matrix([lvx, lvy])
l = Matrix([lp, lv, [ltheta], [lomega]])
l.T

[λₓ  λ_y  λᵥ ₓ  λ_v_y  λ_θ  λ_ω]

In [4]:
# fullstate
fs = Matrix([s, l])
fs.T

[x  y  vx  vy  θ  ω  λₓ  λ_y  λᵥ ₓ  λ_v_y  λ_θ  λ_ω]

In [5]:
# params: gravity, max thrust, mass, half length of auv
g, T, m, length, alpha, rho, cd, A = symbols('g T m L alpha rho C_d A', real=True, positive=True)
params = Matrix([g, T, m, length, alpha, rho, cd, A])
params.T

[g  T  m  L  α  ρ  C_d  A]

In [22]:
# actions
ut = symbols('u_t', real=True, positive=True)
ux, uy = symbols('u_x u_y', real=True)
u = Matrix([ut, ux, uy])
u.T

[u_t  uₓ  u_y]

In [23]:
# thrust vector tilt wrt global frame
#t = Matrix([cos(theta + phi), sin(theta + phi)])
t = Matrix([ux, uy])

# rocket orientation wrt horizontal
itheta = Matrix([cos(theta), sin(theta)])

# perpendicular direction to orientation
itau = Matrix([sin(theta), -cos(theta)])

In [24]:
# dynamics
dr = v
dv = T*ut*t + Matrix([0, -g]) + Rational(1, 2)*rho*sqrt(sum([vec**2 for vec in v]))*v*cd*A
dtheta = omega
domega = (12/m/length**2) * T*ut*length/2 * t.dot(itau)
ds = Matrix([dr, dv, [dtheta], [domega]])
ds

⎡                   vx                    ⎤
⎢                                         ⎥
⎢                   vy                    ⎥
⎢                                         ⎥
⎢                ___________              ⎥
⎢               ╱   2     2               ⎥
⎢  A⋅C_d⋅ρ⋅vx⋅╲╱  vx  + vy                ⎥
⎢  ───────────────────────── + T⋅u_t⋅uₓ   ⎥
⎢              2                          ⎥
⎢                                         ⎥
⎢              ___________                ⎥
⎢             ╱   2     2                 ⎥
⎢A⋅C_d⋅ρ⋅vy⋅╲╱  vx  + vy                  ⎥
⎢───────────────────────── + T⋅u_t⋅u_y - g⎥
⎢            2                            ⎥
⎢                                         ⎥
⎢                    ω                    ⎥
⎢                                         ⎥
⎢    6⋅T⋅u_t⋅(uₓ⋅sin(θ) - u_y⋅cos(θ))     ⎥
⎢    ────────────────────────────────     ⎥
⎣                  L⋅m                    ⎦

In [25]:
# dynamic Lagrangian to minimise thrust usage
L = (1- alpha)*ut**2 + alpha*ut
L

           2         
α⋅u_t + u_t ⋅(-α + 1)

In [26]:
# Hamiltonian
H = l.dot(ds) + L
H

                     ⎛              ___________           ⎞         ⎛         
                     ⎜             ╱   2     2            ⎟         ⎜         
                     ⎜A⋅C_d⋅ρ⋅vx⋅╲╱  vx  + vy             ⎟         ⎜A⋅C_d⋅ρ⋅v
α⋅u_t + λ_θ⋅ω + λᵥ ₓ⋅⎜───────────────────────── + T⋅u_t⋅uₓ⎟ + λ_v_y⋅⎜─────────
                     ⎝            2                       ⎠         ⎝         

     ___________                ⎞                                             
    ╱   2     2                 ⎟                                             
y⋅╲╱  vx  + vy                  ⎟                       2            6⋅T⋅λ_ω⋅u
──────────────── + T⋅u_t⋅u_y - g⎟ + λₓ⋅vx + λ_y⋅vy + u_t ⋅(-α + 1) + ─────────
   2                            ⎠                                             

                           
                           
_t⋅(uₓ⋅sin(θ) - u_y⋅cos(θ))
───────────────────────────
       L⋅m                 

In [27]:
# optimal thrust direction
laux = (lv + lomega*12/length**2*itau)
lauxnorm = sqrt(sum([laux[i]**2 for i in range(len(laux))]))
lauxdir = laux/lauxnorm
tstar = -lauxdir
tstar = simplify(tstar)
tstar

⎡                  ⎛ 2                     ⎞                   ⎤
⎢                 -⎝L ⋅λᵥ ₓ + 12⋅λ_ω⋅sin(θ)⎠                   ⎥
⎢──────────────────────────────────────────────────────────────⎥
⎢    __________________________________________________________⎥
⎢   ╱                          2                             2 ⎥
⎢  ╱  ⎛ 2                     ⎞    ⎛ 2                      ⎞  ⎥
⎢╲╱   ⎝L ⋅λᵥ ₓ + 12⋅λ_ω⋅sin(θ)⎠  + ⎝L ⋅λ_v_y - 12⋅λ_ω⋅cos(θ)⎠  ⎥
⎢                                                              ⎥
⎢                     2                                        ⎥
⎢                  - L ⋅λ_v_y + 12⋅λ_ω⋅cos(θ)                  ⎥
⎢──────────────────────────────────────────────────────────────⎥
⎢    __________________________________________________________⎥
⎢   ╱                          2                             2 ⎥
⎢  ╱  ⎛ 2                     ⎞    ⎛ 2                      ⎞  ⎥
⎣╲╱   ⎝L ⋅λᵥ ₓ + 12⋅λ_ω⋅sin(θ)⎠  + ⎝L ⋅λ_v_y - 12⋅λ_ω⋅cos(θ)⎠  ⎦

In [28]:
# new Hamiltonian with optimal thrust direction
drs = v
dvs = T*ut*tstar + Matrix([0, -g]) + Rational(1, 2)*rho*sqrt(sum([vec**2 for vec in v]))*v*cd*A
dthetas = omega
domegas = 12*T*ut/m/length**2 * tstar.dot(itau)
dss = Matrix([drs, dvs, [dthetas], [domegas]])
Hstar = l.dot(dss) + L
Hstar

                                                                              
                                                                              
                                                                              
                                                                              
                     ⎛              ___________                               
                     ⎜             ╱   2     2                         ⎛ 2    
                     ⎜A⋅C_d⋅ρ⋅vx⋅╲╱  vx  + vy                    T⋅u_t⋅⎝L ⋅λᵥ 
α⋅u_t + λ_θ⋅ω + λᵥ ₓ⋅⎜───────────────────────── - ────────────────────────────
                     ⎜            2                   ________________________
                     ⎜                               ╱                        
                     ⎜                              ╱  ⎛ 2                    
                     ⎝                            ╲╱   ⎝L ⋅λᵥ ₓ + 12⋅λ_ω⋅sin(θ

                                                   

In [29]:
# optimal thrust magnitude (unbounded)
utstar = solve(Hstar.diff(ut), ut)[0]
utstar = simplify(utstar)
utstar

                                               _______________________________
 6       2      6        2      4       2     ╱  4     2    4      2       2  
L ⋅α⋅λᵥ ₓ ⋅m + L ⋅α⋅λ_v_y ⋅m - L ⋅T⋅λᵥ ₓ ⋅m⋅╲╱  L ⋅λᵥ ₓ  + L ⋅λ_v_y  + 24⋅L ⋅λ
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              
                                                                              

___________________________________________________                    _______
                     2                           2     4        2     ╱  4    
_ω⋅λᵥ ₓ⋅sin(θ) - 24⋅L ⋅λ_ω⋅λ_v_y⋅cos(θ) + 144⋅λ_ω   - L ⋅T⋅λ_v_y ⋅m⋅╲╱  L ⋅λᵥ 
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                   

In [31]:
ustar = Matrix([[utstar], tstar])
ustar

⎡                                               ______________________________
⎢ 6       2      6        2      4       2     ╱  4     2    4      2       2 
⎢L ⋅α⋅λᵥ ₓ ⋅m + L ⋅α⋅λ_v_y ⋅m - L ⋅T⋅λᵥ ₓ ⋅m⋅╲╱  L ⋅λᵥ ₓ  + L ⋅λ_v_y  + 24⋅L ⋅
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                   

In [32]:
# costate equations of motion
dl = -Matrix([H.diff(var) for var in s])

# fullstate
dfs = Matrix([ds, dl])
dfs = simplify(dfs)
dfs

⎡                                    vx                                     ⎤
⎢                                                                           ⎥
⎢                                    vy                                     ⎥
⎢                                                                           ⎥
⎢                                 ___________                               ⎥
⎢                                ╱   2     2                                ⎥
⎢                   A⋅C_d⋅ρ⋅vx⋅╲╱  vx  + vy                                 ⎥
⎢                   ───────────────────────── + T⋅u_t⋅uₓ                    ⎥
⎢                               2                                           ⎥
⎢                                                                           ⎥
⎢                               ___________                                 ⎥
⎢                              ╱   2     2                                  ⎥
⎢                 A⋅C_d⋅ρ⋅vy⋅╲╱  vx  + vy                       

In [33]:
# fullstate Jacobian
ddfs = dfs.jacobian(fs)
ddfs

⎡0  0                                                                   1     
⎢                                                                             
⎢0  0                                                                   0     
⎢                                                                             
⎢                                                                             
⎢                                                               2             
⎢                                                     A⋅C_d⋅ρ⋅vx       A⋅C_d⋅ρ
⎢0  0                                               ──────────────── + ───────
⎢                                                        ___________          
⎢                                                       ╱   2     2           
⎢                                                   2⋅╲╱  vx  + vy            
⎢                                                                             
⎢                                                   

In [37]:
u

⎡u_t⎤
⎢   ⎥
⎢uₓ ⎥
⎢   ⎥
⎣u_y⎦

# Code

In [54]:
# function names
names = ["lagrangian", "hamiltonian", "pontryagin", "eom_fullstate", "eom_fullstate_jac"]

# function expressions
funcs = [
    Eq(symbols("L"), L),
    Eq(symbols("H"), H),
    Eq(MatrixSymbol("u", *ustar.shape), ustar),
    Eq(MatrixSymbol("dfs", *dfs.shape), dfs),
    Eq(MatrixSymbol("ddfs", *ddfs.shape), ddfs)
]

# function arguments
syms = [
    [*u, alpha],
    [*fs, *u, alpha, *params],
    [*fs, alpha, *params],
    [*fs, *u, *params],
    [*fs, *u, *params]
]
[sym.append(func.lhs) for sym, func in zip(syms, funcs)]

SyntaxError: invalid syntax (<ipython-input-54-ac4e70362428>, line 15)

In [ ]:
pprint(dfs)

In [ ]:
dfs
